In [54]:
import pandas as pd
import numpy as np
import feather
import sqlite3

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession 

pd.options.display.max_columns = None

# Basic SQL

In [11]:
con = sqlite3.connect('tickets.db')
cur = con.cursor()

In [73]:
violation_count =  pd.read_sql_query("Select ViolationCode,ViolationDescription, Count(*) AS ViolationCount FROM Parking_Violations_2015 GROUP BY ViolationCode",con)

In [74]:
violation_count.sort_values(by='ViolationCount',ascending=False).head(20)

,ViolationCode,ViolationDescription,ViolationCount
21,21,21-No Parking (street clean),1630912
38,38,38-Failure to Display Muni Rec,1418627
14,14,14-No Standing,988469
36,36,PHTO SCHOOL ZN SPEED VIOLATION,839197
37,37,37-Expired Muni Meter,795918
7,7,None,719753
20,20,20A-No Parking (Non-COM),662904
46,46,46A-Double Parking (Non-COM),595709
71,71,71A-Insp Sticker Expired (NYS),576940
40,40,40-Fire Hydrant,534447


Looks like we don't need to download all the rows of the data - some of the ViolationCodes aren't related to pick-ups - but descriptions aren't clean so downloading violation codes for NYC OpenData

In [75]:
violation_codes =pd.read_json("https://data.cityofnewyork.us/resource/ncbg-6agr.json")
violation_codes

,code,definition,manhattan_96th_st_below,all_other_areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,50,50
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,65,65
93,96,RAILROAD CROSSING,95,95
94,97,VACANT LOT,65,45
95,98,OBSTRUCTING DRIVEWAY,95,95


In [76]:
# get map definitions from ViolationCode

violation_count['definition']=""
#violation_count.set_index('ViolationCode',inplace=True)
violation_codes.set_index('code',inplace=True)
violation_count.update(violation_codes)
#violation_count.reset_index(drop=True,inplace=True)
violation_codes.reset_index(drop=True,inplace=True)

In [90]:
violation_count['%_of_total']=violation_count['ViolationCount']/violation_count['ViolationCount'].sum()
violation_count.sort_values(by='ViolationCount',ascending=False).head(30)

,ViolationCode,ViolationDescription,ViolationCount,definition,%_of_total
21,21,21-No Parking (street clean),1630912,NO PARKING-STREET CLEANING,0.138105
38,38,38-Failure to Display Muni Rec,1418627,FAIL TO DSPLY MUNI METER RECPT,0.120129
14,14,14-No Standing,988469,NO STANDING-DAY/TIME LIMITS,0.083703
36,36,PHTO SCHOOL ZN SPEED VIOLATION,839197,PHTO SCHOOL ZN SPEED VIOLATION,0.071063
37,37,37-Expired Muni Meter,795918,EXPIRED MUNI METER,0.067398
7,7,None,719753,FAILURE TO STOP AT RED LIGHT,0.060948
20,20,20A-No Parking (Non-COM),662904,NO PARKING-DAY/TIME LIMITS,0.056134
46,46,46A-Double Parking (Non-COM),595709,DOUBLE PARKING,0.050444
71,71,71A-Insp Sticker Expired (NYS),576940,INSP. STICKER-EXPIRED/MISSING,0.048855
40,40,40-Fire Hydrant,534447,FIRE HYDRANT,0.045257


# Load Datasets and Preprocess

We load the following datasets for use:
1. Parking_Violations_Issued_-_Fiscal_Year_2020.csv
2. Parking_Violations_Issued_-_Fiscal_Year_2021.csv

In [42]:
pluto = pd.read_csv('/Users/stuartong/uberticketsyelp/Data/nyc_pluto_21v1_csv/pluto_21v1.csv', low_memory=False)
ticket_2020 = pd.read_csv('/Users/stuartong/uberticketsyelp/Data/tickets/Parking_Violations_Issued_-_Fiscal_Year_2020.csv', low_memory=False)
ticket_2019 = pd.read_csv('/Users/stuartong/uberticketsyelp/Data/tickets/Parking_Violations_Issued_-_Fiscal_Year_2019.csv', low_memory=False)

We use Feather to cut down the filesize and load times as csv files are large

In [47]:
feather.write_dataframe(pluto,'pluto.feather')
feather.write_dataframe(ticket_2020,'ticket_2020.feather')
feather.write_dataframe(ticket_2019,'ticket_2019.feather')

In [49]:
pluto_df = feather.read_dataframe('pluto.feather')
ticket20_df = feather.read_dataframe('ticket_2020.feather')
ticket19_df = feather.read_dataframe('ticket_2019.feather')

In [53]:
pluto.columns

Index(['borough', 'block', 'lot', 'cd', 'ct2010', 'cb2010', 'schooldist',
       'council', 'zipcode', 'firecomp', 'policeprct', 'healthcenterdistrict',
       'healtharea', 'sanitboro', 'sanitdistrict', 'sanitsub', 'address',
       'zonedist1', 'zonedist2', 'zonedist3', 'zonedist4', 'overlay1',
       'overlay2', 'spdist1', 'spdist2', 'spdist3', 'ltdheight', 'splitzone',
       'bldgclass', 'landuse', 'easements', 'ownertype', 'ownername',
       'lotarea', 'bldgarea', 'comarea', 'resarea', 'officearea', 'retailarea',
       'garagearea', 'strgearea', 'factryarea', 'otherarea', 'areasource',
       'numbldgs', 'numfloors', 'unitsres', 'unitstotal', 'lotfront',
       'lotdepth', 'bldgfront', 'bldgdepth', 'ext', 'proxcode', 'irrlotcode',
       'lottype', 'bsmtcode', 'assessland', 'assesstot', 'exempttot',
       'yearbuilt', 'yearalter1', 'yearalter2', 'histdist', 'landmark',
       'builtfar', 'residfar', 'commfar', 'facilfar', 'borocode', 'bbl',
       'condono', 'tract2010', 'xcoo

In [57]:
pluto_head=['borough','borocode','cd',
            'block', 'zipcode','address',
            'xcoord','ycoord','latitude',
            'longitude'
           ]

# The XY coordinates are expressed in the New York-Long Island State Plane coordinate system.

In [74]:
pluto_df=pluto_df[pluto_head]
pluto_df[pluto_df['address'].str.contains('3RD AVENUE',na=False)]

,borough,borocode,cd,block,zipcode,address,xcoord,ycoord,latitude,longitude
13597,BX,2,201.0,2319,10451.0,2457 3RD AVENUE,1003627.0,234180.0,40.809424,-73.930003
97412,QN,4,408.0,6682,11367.0,147-22 73RD AVENUE,1034756.0,203571.0,40.725288,-73.817783
165015,BX,2,201.0,2320,10451.0,2505 3RD AVENUE,1004025.0,234562.0,40.810472,-73.928564
268512,MN,1,106.0,891,10016.0,534 3RD AVENUE,990428.0,211379.0,40.746861,-73.977704
270410,MN,1,108.0,1510,10028.0,1444 3RD AVENUE,996363.0,222095.0,40.776267,-73.956265
597900,BX,2,203.0,2371,10456.0,3401 3RD AVENUE,1009860.0,241125.0,40.828471,-73.907460
638617,BX,2,201.0,2319,10451.0,2447 3RD AVENUE,1003568.0,234178.0,40.809419,-73.930216
731538,BK,3,310.0,6133,11209.0,9952 3RD AVENUE,974288.0,162765.0,40.613423,-74.035881
794701,BX,2,206.0,2924,10457.0,4179 3RD AVENUE,1012766.0,247567.0,40.846143,-73.896932
797639,MN,1,111.0,1784,10035.0,2201 3RD AVENUE,1001436.0,230831.0,40.800237,-73.937926


## Function to read address from Tickets and look up Pluto for Lat Long

In [121]:
def get_latlongs(input_string):
    '''Function uses Pluto DF to return zips and lat longs'''
    replace_dict = {'ST':'STREET','AVE':'AVENUE'}
    print(input_string)
    # First check input string for short forms i.e. st, ave etc
    if input_string==np.nan:
        return 'No address'
    else:
    # First check input string for short forms i.e. st, ave etc
        input_list = input_string.upper().split()
        updated_list = [i if i not in replace_dict else replace_dict[i] for i in input_list]
        add = ' '.join(updated_list)
        lat = pluto_df.loc[(pluto_df['address']==add),'latitude']
        long = pluto_df.loc[(pluto_df['address']==add),'longitude']
        if len(lat)==0:
            return 'No match found'
        elif len(lat)>1:
            return 'More than add match'
        else:
            return lat.iloc[0],long.iloc[0]

get_latlongs('1953 UNION ST')

1953 UNION ST


(40.667599100000004, -73.9235623)

In [122]:
ticket20_df['address']=ticket20_df['House Number'].astype(str)+' '+ticket20_df['Street Name']

In [128]:
ticket20_df.dropna(subset=['address'],inplace=True)

In [ ]:
ticket20_df['latlong']=ticket20_df['address'].apply(lambda x: get_latlongs(x))

270 43 ST
1953 UNION ST
45 CLERMONT AVENUE
218 DIVISION AVE
850 GRAND ST
None B 99 ST
None W/S/O WASHINGTON ST
2734 BAINBRIDGE AVE
372 94 ST
65 E 54 ST
985 ROCKAWAY AVE
283 E 167 ST
985 ROCKAWAY AVE
985 ROCKAWAY AVE
252 MARTENSE ST
985 ROCKAWAY AVE
200 E 58 ST
541 W 125 ST
1085 E 40 STREET
156 44TH ST
348 E 44
302 8TH AVE
470 PARK AVE
203 MACON ST
19-15 SEAGIRT BLVD
159 W 16 STREET
275 1ST AVE
222 E 52
None N/S E 52 ST
45 WADSWORTH TERRACE
None W/S STEWARD AVE
458 9TH AVE
97-26 99 ST
224 W 30 ST
None E 53 STREET
6402 7 AVENUE
1413 HUGH L GRANT HIGHWAY
55 W 137TH STREET
15 VERMILYEA AVE
40-11 BELL BLVD
121 W 179 ST
112-20 ROCKAWAY BLVD
390 WADSWORTH AVE
241 W 42 ST
1381 UTICA AVE
2304 86 STREET
864 6TH AVENUE
496 FRANKLIN AVE
400 E 23 ST
111 CENTRE ST
215-15 67 AVE
779 45 ST
1624 SAINT NICHOLAS AVE
645 W 160 ST
748 SHEPHARD AVE
None S/W/C SNEDIKER
7030 PERRY TERRACE
742 E 103 ST
12 LENOX AVENUE
45 80 BROADWAY
71-33 CALDWELL AVE
108-02 ROOSEVELT AVE
5745 47 STREET
600 W 196 ST
1137 BRIGH

95 Hanson Pl
650 Fulton St
504 Myrtle Ave
481 Myrtle Ave
100 Steuben St
70 Steuben St
504 Myrtle Ave
930 Halsey St
1285 Broadway
63 Malcolm X Blvd
764 Manhattan Ave
802 Manhattan Ave
922 Manhattan Ave
S Calyer St
155 Milton St
887 Manhattan Ave
94 Kent St
1077 Manhattan Ave
1055 Manhattan Ave
1018 Manhattan Ave
940 Manhattan Ave
292 Grand St
140 Havemeyer St
56 S 10th St
97 Division Ave
113 Clymer St
293 Hewes St
830 Manhattan Ave
141 India St
136 Greenpoint Ave
94 Greenpoint Ave
112 Meserole Ave
738 Manhattan Ave
650 Manhattan Ave
155 N 3rd St
68 Graham Ave
118 McKibbin St
106 Moore St
109 Moore St
91 Moore St
7 Graham Ave
97 Seigel St
926 Manhattan Ave
W India St
191 Calyer St
191 Calyer St
1012 Manhattan Ave
N Manhattan Ave
123 Greenpoint Ave
117 Greenpoint Ave
114 Franklin St
141 Noble St
150 Noble St
W Noble St
836 Lorimer St
84 Manhattan Ave
94 Manhattan Ave
100 McKibbin St
102 Moore St
44 Graham Ave
12 Graham Ave
7 Debevoise St
7 Debevoise St
29 Moore St
599 Manhattan Ave
708 Ma

50 Parkside Ave
50 Parkside Ave
None N/E C/O PERRY
S S 5th St
146 Broadway
52 Broadway
425 Wythe Ave
422 Bedford Ave
53 Broadway
168 7th St
157 2nd Ave
724 51st St
S 50th St
W 9th Ave
779 40th St
770 5th Ave
W 2nd Ave
W 2nd Ave
S 43rd St
S 47th St
1126 Manhattan Ave
127 Dupont St
1054 Manhattan Ave
162 India St
212 Kent St
136 Freeman St
136 Freeman St
66 Freeman St
99 Kent St
120 Kent St
65 Washington St
25 Washington St
72 Front St
S Plymouth St
9 Evans St
185 York St
185 York St
186 Montague St
153 Dekalb Ave
72 S Portland Ave
33 S Portland Ave
11 S Portland Ave
S Dekalb Ave
75 Livingston St
2116 Dorchester Rd
2105 Cortelyou Rd
1919 Cortelyou Rd
1601 Cortelyou Rd
1617 Cortelyou Rd
W E 17th St
1204 Coney Island Ave
1018 Coney Island Ave
1018 Coney Island Ave
50 Parkside Ave
1806 Caton Ave
200 N 11th St
13 Newel St
46 Broome St
164 Bayard St
140 Bayard St
143 Bayard St
488 Graham Ave
140 Newton St
515 Graham Ave
555 Graham Ave
238 Nassau Ave
125 Russell St
583 Leonard St
702 Lorimer S

62 Water St
50 Jay St
17 Monitor St
26 Conselyea St
32 Conselyea St
275 Ainslie St
255 Ainslie St
193 Ainslie St
183 Ainslie St
81 Powers St
113 Powers St
147 Powers St
155 Powers St
147 Powers St
155 Powers St
110 Ainslie St
91 Graham Ave
750 Grand St
706 Grand St
N Eastern Pky
855 Louisiana Ave
E Ardsley Loop
50 Vandalia Ave
W Pennsylvania Ave
2053 Pitkin Ave
2227 Pitkin Ave
N Myrtle Ave
195 Dekalb Ave
190 Hall St
172 Hall St
140 St Edwards St
W Flatbush Ave Exten
S Flushing Ave
S Front St
52 Pearl St
21 Monitor St
229 Ainslie St
177 Ainslie St
182 Ainslie St
15 Powers St
15 Powers St
69 Powers St
112 Powers St
141 Powers St
187 Powers St
187 Powers St
240 Powers St
529 Grand St
559 Grand St
612 Grand St
518 Grand St
N Manhattan Ave
N Eastern Pky
S Linden Blvd
N Stanley Ave
W Van Siclen Ave
50 Vandalia Ave
50 Vandalia Ave
300 Georgia Ave
651 Sutter Ave
195 Dekalb Ave
S Myrtle Ave
W Flatbush Ave Exten
S Concord St
S Concord St
E Bridge Plaza Ct
W Schenectady Ave
E E 48th St
68 Jay St


118 Graham Ave
S Seigel St
13 Marcus Garvey Blvd
E Marcus Garvey Blvd
22 Sumner Pl
500 Dekalb Ave
519 Dekalb Ave
521 Dekalb Ave
300 Lincoln Ave
275 Lincoln Ave
272 Lincoln Ave
70 Lincoln Ave
3024 Fulton St
141 Barbey St
132 Jackie Robinson Pkwy
65 Bulwer Pl
154 Highland Blvd
252 Broadway
229 Havemeyer St
224 S 3rd St
238 S 3rd St
172 Havemeyer St
189 S 9th St
125 Taylor St
272 S 5th St
282 S 5th St
80 Heyward St
S Heyward St
166 Wallabout St
S Wallabout St
29 Gallatin Pl
195 Montague St
195 Montague St
E Red Hook Ln
E Red Hook Ln
E Red Hook Ln
1 Hoyt St
1 Hoyt St
365 State St
422 State St
815 St Johns Pl
W Franklin Ave
210 Joralemon St
384 State St
29 Gallatin Pl
29 Gallatin Pl
S St Johns Pl
10 Beaver St
818 Flushing Ave
424 Bushwick Ave
E Ellery St
S Beaver St
797 Flushing Ave
265 Skillman St
66 Lincoln Ave
42 Lincoln Ave
53 Lincoln Ave
W Lincoln Ave
76 Nichols Ave
159 Nichols Ave
179 Nichols Ave
213 Nichols Ave
142 Hale Ave
3028 Fulton St
2972 Fulton St
124 Sunnyside Ave
310 Highland

...

None EB E 163RD ST @ SIMP
None EB E 163RD ST @ FOX
None SB NOSTRAND AVE @ LI
None NB WEBSTER AVE @ E F
None EB E 163RD ST @ FOX
None SB WEBSTER AVE @ E 1
None NB ROGERS AVE @ EAST
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None SB UTICA AVE @ WINTH
None SB UTICA AVE @ EMPIR
None SB NOSTRAND AVE @ TI
None NB UTICA AVE @ LENOX
None EB KINGS HIGHWAY @ E
None SB WEBSTER AVE @ E 1
None SB WOODHAVEN BLVD @
None EB E 163RD ST @ SIMP
None NB ROGERS AVE @ MONT
None SB NOSTRAND AVE @ TI
None NB WOODHAVEN BLVD @
None WB 207TH ST @ VERMIL
None EB E 23RD ST @ 2ND A
None SB NOSTRAND AVE @ LI
None SB UTICA AVE @ EMPIR
None EB E 163RD ST @ SIMP
None SB NOSTRAND AVE @ LI
None NB ROGERS AVE @ MONT
None EB E 163RD ST @ SIMP
None SB 2ND AVE @ E 85TH
None NB UTICA AVE @ EASTE
None NB WEBSTER AVE @ E 1
None SB NOSTRAND AVE @ LI
None NB WOODHAVEN BLVD @
None EB E 163RD ST @ FOX
None SB 2ND AVE @ E 63RD
None WB 207TH ST @ VERMIL
None SB 2ND AVE @ E 

None EB E 161ST ST @ GERA
None NB WEBSTER AVE @ E 1
None NB WEBSTER AVE @ E 1
None EB FORDHAM RD @ ANDR
None WB KINGS HIGHWAY @ E
None WB KINGS HIGHWAY @ E
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None WB KINGS HIGHWAY @ E
None SB WEBSTER AVE @ BED
None EB ARCHER AVE @ 160T
None NB WEBSTER AVE @ E 1
None NB WOODHAVEN BLVD @
None SB UTICA AVE @ AVE N
None EB E 161ST ST @ GERA
None EB E 161ST ST @ GERA
None NB MAIN ST @ FRANKLI
None WB E 161ST ST @ SHER
None NB 1ST AVE @ ST MARK
None SB 2ND AVE @ E 45TH
None SB UTICA AVE @ AVE N
None EB E 161ST ST @ GERA
None NB 1ST AVE @ ST MARK
None EB 125TH ST @ PARK A
None NB 1ST AVE @ E 106TH
None NB MAIN ST @ FRANKLI
None EB E 163RD ST @ FOX
None SB WEBSTER AVE @ E 2
None WB E 161ST ST @ SHER
None NB 1ST AVE @ E 23RD
None EB E 161ST ST @ GERA
None EB 125TH ST @ 2ND AV
None NB WEBSTER AVE @ E 1
None WB E 161ST ST @ SHER
None NB 1ST AVE @ E 62ND
None SB 2ND AVE @ E 96TH
None SB 2ND AVE @ E 79TH
None SB NOSTRAND AVE @ EA
None SB 2ND AVE @ E

None WB E 34TH ST @ MADIS
None EB E 163RD ST @ SIMP
None NB ROGERS AVE @ ERAS
None SB UTICA AVE @ WINTH
None EB E 163RD ST @ FOX
None SB 2ND AVE @ E 57TH
None NB UTICA AVE @ AVE L
None NB UTICA AVE @ LENOX
None NB UTICA AVE @ AVE L
None NB 1ST AVE @ ST MARK
None SB NOSTRAND AVE @ LI
None NB 1ST AVE @ ST MARK
None WB KINGS HIGHWAY @ E
None EB FORDHAM RD @ WASH
None WB 125TH ST @ ADAM C
None NB ROGERS AVE @ ERAS
None SB WOODHAVEN BLVD @
None WB E 161ST ST @ SHER
None EB E 161ST ST @ GERA
None SB UTICA AVE @ WINTH
None NB WOODHAVEN BLVD @
None SB WEBSTER AVE @ E 2
None EB E 163RD ST @ SIMP
None SB NOSTRAND AVE @ LI
None SB NOSTRAND AVE @ TI
None NB 1ST AVE @ E 86TH
None EB E 161ST ST @ GERA
None SB UTICA AVE @ EMPIR
None EB 125TH ST @ 5TH AV
None EB E FORDHAM RD @ GR
None SB UTICA AVE @ EMPIR
None WB 125TH ST @ ADAM C
None EB E 161ST ST @ GERA
None EB E 161ST ST @ GERA
None EB E 163RD ST @ SIMP
None WB E 161ST ST @ SHER
None SB UTICA AVE @ WINTH
None EB KINGS HIGHWAY @ E
None SB MAIN ST @

None SB WOODHAVEN BLVD @
None WB E 161ST ST @ SHER
None SB WEBSTER AVE @ E 1
None EB ARCHER AVE @ 160T
None WB E 161ST ST @ SHER
None EB W 23RD ST @ 9TH A
None EB E 163RD ST @ THIR
None SB NOSTRAND AVE @ LI
None SB WEBSTER AVE @ E 2
None EB E 23RD ST @ 2ND A
None NB UTICA AVE @ DEAN
None SB WOODHAVEN BLVD @
None EB E FORDHAM RD @ GR
None WB KINGS HIGHWAY @ E
None WB E 161ST ST @ SHER
None NB ROGERS AVE @ ERAS
None WB E 161ST ST @ SHER
None EB E FORDHAM RD @ GR
None NB UTICA AVE @ SNYDE
None SB 2ND AVE @ E 61ST
None WB KINGS HIGHWAY @ M
None SB 2ND AVE @ E 63RD
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None NB UTICA AVE @ LENOX
None SB WOODHAVEN BLVD @
None SB 2ND AVE @ E 34TH
None NB 1ST AVE @ E 86TH
None SB WOODHAVEN BLVD @
None SB 2ND AVE @ E 96TH
None WB E 161ST ST @ SHER
None WB E 161ST ST @ SHER
None EB E 163RD ST @ FOX
None SB NOSTRAND AVE @ LI
None SB 2ND AVE @ E 96TH
None EB FORDHAM RD @ ANDR
None SB WOODHAVEN BLVD @
None SB UTICA AVE @ WINTH

None ASTORIA BLVD (E/B) @
None HORACE HARDING EXPWY
None MCDONALD AVE (S/B) @
None NORTHERN BLVD (W/B)
None CONEY ISLAND AVE (S/
None FATHER CAPODANNO BLV
None BRUCKNER BLVD (E/B)
None PENNSYLVANIA AVE (N/
None ASTORIA BLVD (W/B) @
None CROPSEY AVE (W/B) @
None ASTORIA BLVD (E/B) @
None BEACH CHANNEL DR (S/
None 3RD AVE (N/B) @ 36TH
None HORACE HARDING EXPWY
None BAY ST (S/B) @ HANNA
None SEAGIRT BLVD (W/B) @
None ASTORIA BLVD S (E/B)
None PENNSYLVANIA AVE (N/
None EASTERN PKWY (W/B) @
None SO CONDUIT AVE (E/B)
None ROCKAWAY BLVD (S/B)
None HORACE HARDING EXPWY
None ASTORIA BLVD S (E/B)
None EB GRAND CENTRAL PKW
None MCDONALD AVE (S/B) @
None 222ND ST (E/B) @ PAU
None W FORDHAM RD (W/B) @
None LINDEN BLVD (W/B) @
None EB GRAND CENTRAL PKW
None WOODHAVEN BLVD (N/B)
None RICHMOND RD (E/B) @
None BROADWAY (S/B) @ SHE
None BRONX RIVER PKWY OFF
None ASTORIA BLVD (E/B) @
None HORACE HARDING EXPWY
None ATLANTIC AVE (E/B) @
None NB OCEAN PKWY @ 18TH
None WHITESTONE EXPWY SVC
None QUEENS BLVD (

In [127]:
feather.write_dataframe(ticket20_df,'ticket_2020_processed.feather')

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,address
78,1459164362,13460MT,NY,COM,07/17/2000 12:00:00 AM,19,DELV,FORD,P,0,0,0,0,NaN,0,401,956008,0401,0000,0956A,None,Q,F,4161,None,None,0,408,C,None,BBBBBBB,ALL,ALL,None,0.0,0,-,0,None,None,NaN,NaN,NaN,NaN
199,1461416346,92003MH,NY,COM,02/10/2010 12:00:00 AM,46,VAN,ZENI,P,0,0,0,20311231,10.0,10,401,946963,0401,0000,0332P,None,NY,F,365,None,None,0,408,C,None,BBBBBBB,ALL,ALL,YW,0.0,0,-,0,None,None,NaN,NaN,NaN,NaN
260,1403956352,HXD9466,NY,999,07/18/2011 12:00:00 AM,20,P-U,DODGE,X,0,0,0,20200105,122.0,122,992,1172,0992,0000,0726A,None,R,F,None,None,None,0,408,D,None,BBBBBBB,ALL,ALL,BK,0.0,0,-,0,None,None,NaN,NaN,NaN,NaN
262,1452906956,BLANKPLATE,99,999,08/07/2011 12:00:00 AM,41,None,CHEVR,P,0,0,0,20111201,NaN,0,0,0,0000,0000,1000P,None,BX,O,None,None,None,0,408,D,None,BBBBBBB,ALL,ALL,None,0.0,0,-,0,None,None,NaN,NaN,NaN,NaN
2839,1458915736,85390MV,NY,COM,08/01/2018 12:00:00 AM,14,VAN,CHEVR,P,0,0,0,0,17.0,17,401,949733,0401,0000,0819A,None,NY,F,143,None,None,0,408,C,None,BBBBBBB,ALL,ALL,WH,0.0,0,-,0,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12392707,8763181400,JLK6668,NY,PAS,06/19/2020 12:00:00 AM,0,SUBN,LINCO,T,0,0,0,20211029,NaN,0,0,367894,T401,E,0822A,None,None,F,None,None,None,0,408,None,None,YYYYYYY,None,None,GY,NaN,2003,None,0,52-A,None,NaN,NaN,NaN,NaN
12414790,8738265606,RUG1987,NY,PAS,06/22/2020 12:00:00 AM,0,None,None,T,0,0,0,88880088,NaN,0,0,371179,T402,E,0114P,None,None,F,None,None,None,0,408,None,None,None,None,None,None,NaN,0,None,0,SC4,None,NaN,NaN,NaN,NaN
12421291,8848900148,M42LYH,NJ,PAS,06/22/2020 12:00:00 AM,0,4DSD,HONDA,T,0,0,0,88880088,NaN,0,0,372282,T401,T,1040A,None,None,F,None,None,None,0,408,None,None,YYYYYYY,None,None,GOLD,NaN,0,None,0,50-A,None,NaN,NaN,NaN,NaN
12422171,8814073867,L95937MB,NY,COM,06/22/2020 12:00:00 AM,0,None,None,T,0,0,0,88880088,NaN,0,0,355417,T103,K,1016A,None,None,F,None,None,None,0,408,None,None,None,None,None,None,NaN,0,None,0,40,None,NaN,NaN,NaN,NaN


# Pyspark

In [31]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('Milestone1') \
    .getOrCreate()
sc = spark.sparkContext

Exception: Java gateway process exited before sending its port number